In [1]:
"""

query规划：
参考：
https://mirascope.com/tutorials/more_advanced/query_plan/

"""

'\n\nquery规划：\n参考：\nhttps://mirascope.com/tutorials/more_advanced/query_plan/\n\n'

In [2]:
from openai import OpenAI


openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)





In [3]:

from pydantic import BaseModel, Field
from typing import Optional, Annotated, List


class Query(BaseModel):
    # question: str = Field(description="需要做问题分布解答和规划的用户问题")
    # dependencies: list[int] = Field(
    #     description="在回答用户问题之前需要回答的子问题列表",
    # )
    sub_queries: Annotated[List[str], Field(description="用户问题拆分后的有序的子问题列表，子问题列表的答案经过组合可以推理原始问题的答案")]
    # tools: list[str] = Field(description="用来回答问题的工具列表")
    

    




In [4]:
import json

def get_weather_by_year(year: int):
    """虚构数据，获取东京按年的天气数据"""
    if year == 2020:
        data = {
            "一月": 42,
            "二月": 43,
            "三月": 49,
            "四月": 58,
            "五月": 66,
            "六月": 72,
            "七月": 78,
            "八月": 81,
            "九月": 75,
            "十月": 65,
            "十一月": 55,
            "十二月": 47,
        }
    elif year == 2021:
        data = {
            "一月": 45,
            "二月": 48,
            "三月": 52,
            "四月": 60,
            "五月": 68,
            "六月": 74,
            "七月": 80,
            "八月": 83,
            "九月": 77,
            "十月": 67,
            "十一月": 57,
            "十二月": 49,
        }
    else:
        data = {
            "一月": 48,
            "二月": 52,
            "三月": 56,
            "四月": 64,
            "五月": 72,
            "六月": 78,
            "七月": 84,
            "八月": 87,
            "九月": 81,
            "十月": 71,
            "十一月": 61,
            "十二月": 53,
        }
    return json.dumps(data)

In [39]:

"""
定义任务规划：
query plan
"""


def create_query_plan(query):


    system_prompt = """您是一个为问题创建解决方案和步骤的AI专家助手。
    您有一个问题，需要为它创建一个查询规划，该规划包含了回答该问题的查询列表。

    # 您可以使用的工具是：
    # - get_weather_by_year
    # - get_person_name
    """

    system_prompt = """您是一个为问题创建有序解决步骤的AI专家助手，给定一个问题，可以为该问题创建一系列子问题，并且子问题的答案经过推理可以回答原始问题。
    **思考推理**
    - 你需要思考原始问题的解决步骤，并能够产生最终的答案
    - 解决步骤中需要包含原始问题的子问题，每个子问题之间具有连续性和关联性
    - 思考和推理解决问题的步骤要模拟程序员的思维
    - 解决问题的步骤可以参考数据库表设计的逻辑
    
    每个子问题的要求：
    - 子问题列表包含若干个独立的子问题，每个子问题需要可以单独被回答
    - 子问题列表的答案可以推理出回答原始问题的答案
    - 仅回答子问题列表，其它内容不要回答
    - 不要重复子问题
    - 子问题的粒度要适用，不要过于细致
    - 不要产生与原始问题无关的子问题

    # **样例**
    # 原始问题：姚明和张三的身高谁高
    # 子问题列表：['姚明身高是多少', '张三身高是多少', '姚明和张三的身高谁高']
    
    """

    """
    **重点**
    - 检查子问题列表的答案是否可以回答原始问题。
    - 自己要有自我纠错的机制
    - 注意子问题推理的逻辑
    """

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0.001,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                               # "repetition_penalty": 1.05,
                                                       "guide_json": Query.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    
    
    return chat_response.choices[0].message.content



    





In [42]:


def judge_sub_query(query, sub_queries):
    """
    **自我思考推理**
    - 你需要思考原始问题的解决步骤，并能够产生最终的答案
    - 解决步骤中需要包含原始问题的子问题，每个子问题之间具有连续性和关联性
    - 思考和推理解决问题的步骤要模拟程序员的思维
    - 解决问题的步骤可以参考数据库表设计的逻辑

    """

    user_prompt = """ 你是一个问题推理和解决方案的AI专家助手，能够判断出生成的子问题是否能够解答原始问题的答案。
    
    现在给定你一个原始问题和子问题列表，判断是否回答原始问题，你可以删除、添加或者修改子问题列表，使其子问题的答案能够回答原始问题。最终的子问题列表要求：
    - 子问题列表包含若干个独立的子问题，每个子问题需要可以单独被回答
    - 子问题列表的答案可以推理出回答原始问题的答案
    - 仅回答子问题列表，其它内容不要回答
    - 最后一个子问题能够和前面的子问题能够组成最终的答案
    - 保留正确的子问题
    - 子问题之间要层层递进，上一个子问题的答案可能是下一个问题的输入
    - 子问题不要添加无关次，不要随意猜测子问题的答案类型，不要在子问题结尾补充无关次
    - 子问题尽量以名词结束，尽量少使用“多少”、“什么”等词语    
    

    原始问题：{question}
    子问题列表：
    {sub_queries}

    经过原始问题和子问题列表分析后，最终的子问题列表：
    """
    # user_prompt = """ 你是一个问题推理和解决方案的AI专家助手，能够判断出生成的子问题是否能够解答原始问题的答案。
    
    # 现在给定你一个原始问题和子问题列表，判断是否回答原始问题，你可以删除、添加或者修改子问题列表，使其子问题的答案能够回答原始问题。

    # 子问题内容要求：
    # * 子问题列表包含若干个独立的子问题，每个子问题需要可以单独被回答
    # * 子问题列表的答案可以推理出回答原始问题的答案
    # * 子问题之间要层层递进，上一个子问题的答案可能是下一个问题的输入
    # * 子问题尽量以名词结束，不要使用“多少”、“什么”等无关词语
    # * 子问题的分解不要过于细致
    # * 子问题的分解要参考程序员设计数据库表的规范

    # 子问题的格式要求：
    # * 仅回答子问题列表，其它内容不要回答


    

    # 原始问题：{question}
    # 子问题列表：
    # {sub_queries}

    # 最终的子问题列表：
    # """
    system_prompt = "你是一个有益的人工智能助手。"
    prompt = user_prompt.format(**{"question": query, "sub_queries": sub_queries})

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",
                                                   messages=messages, 
                                                   temperature=0.001,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                               "repetition_penalty": 1.05,
                                                       "guide_json": Query.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    return chat_response.choices[0].message.content
    





In [47]:
# query = "功率最大的发电机是什么时候购买的"
# query = "姚明和奥尼尔谁的身高高"
# query = "耗电量最大的设备是什么？为什么它耗电量这么大"
query = "耗电量最大的设备名称是及原因"

# 功率最大的发电机和是什么时候购买的

sub_queries = create_query_plan(query)
print("---------------子问题列表-----------------")
print(sub_queries)
print("---------------最终子问题列表-----------------")
final_sub_queries = judge_sub_query(query, sub_queries)
print(final_sub_queries)


---------------子问题列表-----------------
['耗电量最大的设备名称是什么', '该设备的耗电量是多少', '导致该设备耗电量大的主要原因是什么']
---------------最终子问题列表-----------------
['耗电量最大的设备名称是什么', '导致该设备耗电量大的主要原因是什么']
